## UNet Architecture


In [4]:
import torch
from model import UNet
from dataset import RetinaDataset
from utils import load_data
import torchvision.transforms as transforms
import albumentations as A
import cv2 as cv
import os

In [ ]:
train_images, train_masks, test_images, test_masks = load_data("dataset")
for i in train_images:
    print(i)